# 2. Retriever Node Design

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# data_path
data_path = "./my_weaviate_data"
if not os.path.exists(data_path):
    os.makedirs(data_path)
print(f"Weaviate {data_path}")

Weaviate ./my_weaviate_data


In [3]:
import os
import weaviate

client = weaviate.connect_to_embedded(
    # 1. 버전 정보
    version="1.27.0",
    # 2. 데이터 저장 위치
    persistence_data_path=data_path,
    # 3. API KEY 전달
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")
    },
    # 4. 모듈 활성화
    environment_variables={
        "ENABLE_MODULES": "text2vec-openai",
        "DEFAULT_VECTORIZER_MODULE": "text2vec-openai"
    }
)

if client.is_ready():
    print(f"Weaviate가 준비 되었습니다. {data_path}에 저장됩니다.")
else:
    print("Weaviate 실행에 실패했습니다.")

{"action":"startup","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","default_vectorizer_module":"text2vec-openai","level":"info","msg":"the default vectorizer modules is set to \"text2vec-openai\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2025-12-05T18:59:29+09:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2025-12-05T18:59:29+09:00"}
{"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2025-12-05T18:59:29+09:00"}
{"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","l

Weaviate가 준비 되었습니다. ./my_weaviate_data에 저장됩니다.


{"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"configured versions","server_version":"1.27.0","time":"2025-12-05T18:59:31+09:00","version":"1.27.0"}
{"action":"grpc_startup","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"grpc server listening at [::]:50050","time":"2025-12-05T18:59:31+09:00"}
{"action":"restapi_management","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2025-12-05T18:59:31+09:00","version":"1.27.0"}
{"address":"192.168.123.35:48543","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"current Leader","time":"2025-12-05T18:59:31+09:00"}
{"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg"

{"action":"bootstrap","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"node reporting ready, exiting bootstrap process","time":"2025-12-05T18:59:32+09:00"}
{"action":"telemetry_push","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"telemetry started","payload":"\u0026{MachineID:9e21a580-71c4-4be2-b4de-4755dbda8480 Type:INIT Version:1.27.0 NumObjects:0 OS:linux Arch:amd64 UsedModules:[]}","time":"2025-12-05T18:59:33+09:00"}


In [ ]:
from weaviate.classes.config import Configure, Property, DataType

collection = client.collections.create(
    # 1. Table name
    name="AdvancedRAG_Chunk",
    # 2. Dense Vector 
    vectorizer_config=Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    # 3. Spase Vector
    inverted_index_config=Configure.inverted_index(
        bm25_b=0.75,
        bm25_k1=1.2
    ),
    # 4. Attribute
    properties=[
        Property(name="content", data_type=DataType.TEXT),
        Property(name="doc_id", data_type=DataType.INT)
    ]

)

/mnt/data1/work/sm-ai-v2/my-rag-server/.venv/lib/python3.13/site-packages/weaviate/warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(


{"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"warning","msg":"prop len tracker file my_weaviate_data/advancedrag_chunk/p7GkcsqljZx0/proplengths does not exist, creating new tracker","time":"2025-12-05T18:59:45+09:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2025-12-05T18:59:45+09:00","wait_for_cache_prefill":false}
{"build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","level":"info","msg":"Created shard advancedrag_chunk_p7GkcsqljZx0 in 1.464058ms","time":"2025-12-05T18:59:45+09:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"","build_go_version":"go1.22.0","build_image_tag":"","build_wv_version":"","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector ca

In [5]:
documents = [
    {"content": "Weaviate의 Hybrid Search는 alpha 값을 통해 벡터와 키워드 검색 비중을 조절한다.", "doc_id": 1},
    {"content": "Weaviate는 2024년에 v4 클라이언트를 출시하며 API를 대폭 변경했다.", "doc_id": 2},
    {"content": "ChromaDB는 로컬 파일 기반의 가벼운 벡터 저장소로 빠르게 시작하기 좋다.", "doc_id": 3},
    {"content": "Advanced RAG를 위해서는 Reranker를 사용하여 검색 정확도를 높여야 한다.", "doc_id": 4},
    {"content": "Hybrid Search에서 alpha=1.0이면 순수 벡터 검색, 0.0이면 순수 BM25 검색이다.", "doc_id": 5},
    {"content": "생성형 AI 모델(LLM)은 할루시네이션 문제가 발생할 수 있다.", "doc_id": 6},
]

print("📥 데이터 적재 중...")
with collection.batch.dynamic() as batch:
    for doc in documents:
        batch.add_object(properties=doc)
print("✅ 데이터 적재 완료!")

📥 데이터 적재 중...
✅ 데이터 적재 완료!


In [6]:
from weaviate.classes.query import MetadataQuery

user_query = "RAG성능 고도화의 개념은?"

# Step 4-1: Hybrid Search (Recall 확보 - 넉넉하게 10개)
# 쿼리 리라이팅된 쿼리가 있다면 여기서 반복문으로 여러번 쏘고 합치면 됩니다.
response = collection.query.hybrid(
    query=user_query,
    alpha=0.5, # 밸런스형
    limit=5,  # Reranking을 위해 넉넉히 가져옴
    return_metadata=MetadataQuery(score=True)
)

initial_results = []
for obj in response.objects:
    initial_results.append(obj.properties['content'])
    
print(f"1차 검색 결과: {len(initial_results)}개 추출")

1차 검색 결과: 5개 추출
